# Polyhedral Optimization Evaluation

The Polybench tests were used for doing polyhedral optimization - essentially, trying to improve locality of memory in algebra kernels selected from a range of different fields.  Thus, they are an appropriate tool for studying/analyzing the behavior of PMEM, particularly with respect to its locality.

In late December 2019 I collected a series of data points for Polybench using Intel Optane DC Persistent Memory (Generation 1).  The system that I used deviates from the prior platform because the CPU was replaced.  Thus, I cannot directly compare this data.  This is the same memory, however, so I can still draw reasonable conclusions from this data and make normalized data comparisons (albeit carefully).

```
[tony@localhost polybench-3.2]$ lscpu
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          80
On-line CPU(s) list:             0-79
Thread(s) per core:              2
Core(s) per socket:              20
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Genuine Intel(R) CPU 0000%@
Stepping:                        6
CPU MHz:                         1000.024
CPU max MHz:                     3900.0000
CPU min MHz:                     1000.0000
BogoMIPS:                        5000.00
Virtualization:                  VT-x
L1d cache:                       1.3 MiB
L1i cache:                       1.3 MiB
L2 cache:                        40 MiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0-19,40-59
NUMA node1 CPU(s):               20-39,60-79
Vulnerability L1tf:              Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl and seccomp
Vulnerability Spectre v1:        Mitigation; __user pointer sanitization
Vulnerability Spectre v2:        Mitigation; Enhanced IBRS, IBPB conditional, RSB filling
Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse
                                  sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl
                                 xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg
                                  fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c
                                 rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb cat_l3 cdp_l3 invpcid_single ssbd mba ibrs ibpb stib
                                 p ibrs_enhanced tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 hle avx2 smep b
                                 mi2 erms invpcid rtm cqm mpx rdt_a avx512f avx512dq rdseed adx smap clflushopt clwb intel_pt avx512cd
                                  avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves cqm_llc cqm_occup_llc cqm_mbm_total cqm_mbm_local d
                                 therm ida arat pln pts hwp hwp_act_window hwp_epp hwp_pkg_req pku ospke avx512_vnni flush_l1d arch_ca
                                 pabilities
[tony@localhost polybench-3.2]$ uname -a
Linux intelsdp1044 5.0.0+ #4 SMP Wed Sep 4 15:41:42 EDT 2019 x86_64 x86_64 x86_64 GNU/Linux
```

There are four fewer cores, but the L1, L2, and L3 caches are **substantially** larger.  This will have significant impact on the overall sytem performance.


In [1]:
#!/usr/bin/python3

import os
import sys
import matplotlib.pyplot as plt
import matplotlib.dates as matdates
import tikzplotlib
import numpy as np
import pandas as pd
from collections import OrderedDict
import locale
import statistics
from scipy.stats.mstats import gmean
from cycler import cycler

In [14]:
# first, lets find the interesting directories
dirs = [x for x in os.listdir('.') if 'comerge-pb-poly' in x]
subdirs = {}
for d in dirs:
    subdirs[d] = [x for x in os.listdir(d)]

for sd in subdirs: print('{}: {}\n'.format(sd,subdirs[sd]))

comerge-pb-poly-results-2019_12_28__10_26_54: ['pb-dram2M-intelsdp1044-2019_12_28__10_26_54.log', 'pb-make-intelsdp1044-2019_12_28__10_26_54.log']

comerge-pb-poly-results-2019_12_28__11_19_34: ['pb-dram4K-intelsdp1044-2019_12_28__11_19_34.log', 'pb-make-intelsdp1044-2019_12_28__11_19_34.log', 'pb-pmem3-intelsdp1044-2019_12_28__11_19_34.log', 'pb-pmem4-intelsdp1044-2019_12_28__11_19_34.log']

comerge-pb-poly-results-2019_12_28__12_06_30: ['pb-dram4K-intelsdp1044-2019_12_28__12_06_30.log', 'pb-make-intelsdp1044-2019_12_28__12_06_30.log', 'pb-pmem3-intelsdp1044-2019_12_28__12_06_30.log', 'pb-pmem4-intelsdp1044-2019_12_28__12_06_30.log']

comerge-pb-poly-results-2019_12_28__18_05_52: ['pb-pmem62M-intelsdp1044-2019_12_28__18_05_52.log', 'pb-pmem64K-intelsdp1044-2019_12_28__18_05_52.log']

